# **Alpaca**




In [1]:
!pip install alpaca_trade_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 6.8 MB/s eta 0:00:00
Reason for being yanked: This version includes overly restrictive multidict upper boundary disallowing multidict v6+. The previous patch version didn't have that and this is now causing dependency resolution problems for the users who have an "incompatible" version pinned. This is not really necessary anymore and will be addressed in the next release v3.8.3

https://github.com/aio-libs/aiohttp/pull/6950
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.7
    Uninstalling msgpack-

## **Company Stocks**

### **EMA Trading Strategy**

In [2]:
#Import libraries
import alpaca_trade_api as tradeapi
import numpy as np
import pandas as pd
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#Authentication and connection details
api_key = 'PKSK1QYR2O532NSHT0KU'
api_secret = '7BNhzgktxgEwG8ULs1e8ZOCtDDiWQCgW11RzWwkk'
base_url = 'https://paper-api.alpaca.markets'

#Instantiate REST API
api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')

#Get our account information
account = api.get_account()

#The mail addresses and password
sender_address = 'fintechtraininggroup1@gmail.com'
sender_pass = 'FintechGroup1'
receiver_address = 'fintechtraininggroup1@gmail.com'

#Setup the MIME
message = MIMEMultipart()
message['From'] = 'Trading Bot'
message['To'] = receiver_address
message['Subject'] = 'Fintech Python Trading Bot'

#Selection of stock
symb = "AAPL"
stock_price = []
pos_held = False
mail_content= ""
mail_list = []

#Get market closing price within 10 minutes
def get_market_data():
  minute = 10
  symb = 'AAPL'
  market_data = api.get_barset(symb, 'minute', limit = minute)
  bars = market_data[symb]

  close_list = []
  for bar in market_data[symb]:
      close_list.append(bar.c)

  close_list = np.array(close_list, dtype = np.float64)

  return close_list

#Calculate EMA
def calculate_ema(prices, days, smoothing = 2):
  ema = [sum(prices[:days]) / days]
  for price in prices[days:]:
      ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
  return ema

#Send email
def mailSending(mail_content):
  #The body and the attachments for the mail
  message.attach(MIMEText(mail_content, 'plain'))

  #Create SMTP session for sending the mail
  session = smtplib.SMTP('smtp.gmail.com', 587) #Use gmail with port
  session.starttls() #enable security
  session.login(sender_address, sender_pass) #Login with mail_id and password
  text = message.as_string()
  session.sendmail(sender_address, receiver_address, text)
  session.quit()
  print('Mail Sent')

#Trading algorithm
clock = api.get_clock()
print('The market is {}'.format('open.' if clock.is_open else 'closed.'))

while True:
  print("")
  print("Checking Price")
  ema = calculate_ema(get_market_data(), len(get_market_data()))[-1]
  last_price = get_market_data()[-1] #Most recent closing price
  print("Equity: {0}".format(account.equity))
  print("Exponential Moving Average: " + str(ema))
  print("Last Price: " + str(last_price))

  if clock.is_open == True:
    #Buy
    if last_price > ema and not pos_held: #If price is crossing EMA, and we haven't already bought -> so we buy it
      print("Buy")
      api.submit_order(
        symbol = symb,
        # qty = 1,
        qty = (0.2 * float(account.equity)) // last_price,
        side = 'buy',
        type = 'market',
        time_in_force = 'gtc'
      )
      mail_content = "Trades have been made. You have buy the stocks. \n"
      pos_held = True
      stock_price.append(last_price)
    print(stock_price)

    #Sell
    if len(stock_price) != 0:
      if last_price >= (stock_price[-1] + stock_price[-1] * 0.002) and pos_held:
        print("Sell")
        api.submit_order(
          symbol = symb,
          # qty = 1,
          qty = (0.2 * float(account.equity)) // last_price,
          side = 'sell',
          type = 'market',
          time_in_force = 'gtc'
        )
        mail_content = "Trades have been made. You have sell the stocks and take profit. \n"
        pos_held = False
        stock_price.clear()
      elif last_price <= (stock_price[-1] - stock_price[-1] * 0.005) and pos_held:
        print("Sell")
        api.submit_order(
          symbol = symb,
          # qty = 1,
          qty = (0.2 * float(account.equity)) // last_price,
          side = 'sell',
          type = 'market',
          time_in_force = 'gtc'
        )
        mail_content = "Trades have been made. You have sell the stocks and stop loss. \n"
        pos_held = False
        stock_price.clear()
  else:
    mail_content = "The market is closed. \n"

  #Condition for sending email that should be running only once for each
  if (mail_list.count(mail_content) < 1):
    mailSending(mail_content)
    if (mail_content not in mail_list):
      mail_list.clear() #Clear after send
    mail_list.append(mail_content)  #Append back what we go through

  time.sleep(60) #Refresh every minute

HTTPError: ignored

### **Short Selling (Whole Process)**

In [ ]:
#Import libraries
import datetime
import threading
import alpaca_trade_api as tradeapi
import time
from alpaca_trade_api.rest import TimeFrame

#Authentication and connection details
API_KEY = "PKWNGPNK1B3QSSSWQYNZ"
API_SECRET = "Ny8OOlvzdF57HC3sV6VOd3EM09XuuhT7owIy67W9"
APCA_API_BASE_URL = "https://paper-api.alpaca.markets"


class LongShort:
  def __init__(self):
    self.alpaca = tradeapi.REST(API_KEY, API_SECRET, APCA_API_BASE_URL, 'v2')

    stockUniverse = ['DOMO', 'TLRY', 'SQ', 'MRO', 'AAPL', 'GM', 'SNAP', 'SHOP',
                     'SPLK', 'BA', 'AMZN', 'SUI', 'SUN', 'TSLA', 'CGC', 'SPWR',
                     'NIO', 'CAT', 'MSFT', 'PANW', 'OKTA', 'TWTR', 'TM', 'RTN',
                     'ATVI', 'GS', 'BAC', 'MS', 'TWLO', 'QCOM', ]
    #Format the allStocks variable for use in the class
    self.allStocks = []
    for stock in stockUniverse:
      self.allStocks.append([stock, 0])

    self.long = []
    self.short = []
    self.qShort = None
    self.qLong = None
    self.adjustedQLong = None
    self.adjustedQShort = None
    self.blacklist = set()
    self.longAmount = 0
    self.shortAmount = 0
    self.timeToClose = None

  def run(self):
    #First, cancel any existing orders so they don't impact our buying power.
    orders = self.alpaca.list_orders(status = "open")
    for order in orders:
      self.alpaca.cancel_order(order.id)

    #Wait for market to open
    print("Waiting for market to open...")
    tAMO = threading.Thread(target = self.awaitMarketOpen)
    tAMO.start()
    tAMO.join()
    print("Market opened.")

    #Rebalance the portfolio every minute, making necessary trades
    while True:

      #Figure out when the market will close so we can prepare to sell beforehand
      clock = self.alpaca.get_clock()
      closingTime = clock.next_close.replace(tzinfo = datetime.timezone.utc).timestamp()
      currTime = clock.timestamp.replace(tzinfo = datetime.timezone.utc).timestamp()
      self.timeToClose = closingTime - currTime

      if(self.timeToClose < (60 * 15)):
        #Close all positions when 15 minutes til market close
        print("Market closing soon.  Closing positions.")

        positions = self.alpaca.list_positions()
        for position in positions:
          if(position.side == 'long'):
            orderSide = 'sell'
          else:
            orderSide = 'buy'
          qty = abs(int(float(position.qty)))
          respSO = []
          tSubmitOrder = threading.Thread(target = self.submitOrder(qty, position.symbol, orderSide, respSO))
          tSubmitOrder.start()
          tSubmitOrder.join()

        #Run script again after market close for next trading day
        print("Sleeping until market close (15 minutes).")
        time.sleep(60 * 15)
      else:
        #Rebalance the portfolio
        tRebalance = threading.Thread(target = self.rebalance)
        tRebalance.start()
        tRebalance.join()
        time.sleep(60)

  #Wait for market to open
  def awaitMarketOpen(self):
    isOpen = self.alpaca.get_clock().is_open
    while(not isOpen):
      clock = self.alpaca.get_clock()
      openingTime = clock.next_open.replace(tzinfo=datetime.timezone.utc).timestamp()
      currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
      timeToOpen = int((openingTime - currTime) / 60)
      print(str(timeToOpen) + " minutes til market open.")
      time.sleep(60)
      isOpen = self.alpaca.get_clock().is_open

  def rebalance(self):
    tRerank = threading.Thread(target = self.rerank)
    tRerank.start()
    tRerank.join()

    #Clear existing orders again
    orders = self.alpaca.list_orders(status = "open")
    for order in orders:
      self.alpaca.cancel_order(order.id)

    print("We are taking a long position in: " + str(self.long))
    print("We are taking a short position in: " + str(self.short))
    #Remove positions that are no longer in the short or long list, and make a list of positions that do not need to change. Adjust position quantities if needed.
    executed = [[], []]
    positions = self.alpaca.list_positions()
    self.blacklist.clear()
    for position in positions:
      if(self.long.count(position.symbol) == 0):
        #Position is not in long list
        if(self.short.count(position.symbol) == 0):
          #Position not in short list either. Clear position.
          if(position.side == "long"):
            side = "sell"
          else:
            side = "buy"
          respSO = []
          tSO = threading.Thread(target = self.submitOrder, args = [abs(int(float(position.qty))), position.symbol, side, respSO])
          tSO.start()
          tSO.join()
        else:
          #Position in short list
          if(position.side == "long"):
            #Position changed from long to short. Clear long position to prepare for short position.
            side = "sell"
            respSO = []
            tSO = threading.Thread(target = self.submitOrder, args = [int(float(position.qty)), position.symbol, side, respSO])
            tSO.start()
            tSO.join()
          else:
            if(abs(int(float(position.qty))) == self.qShort):
              #Position is where we want it. Pass for now.
              pass
            else:
              #Need to adjust position amount
              diff = abs(int(float(position.qty))) - self.qShort
              if(diff > 0):
                #Too many short positions. Buy some back to rebalance.
                side = "buy"
              else:
                # Too little short positions. Sell some more.
                side = "sell"
              respSO = []
              tSO = threading.Thread(target = self.submitOrder, args = [abs(diff), position.symbol, side, respSO])
              tSO.start()
              tSO.join()
            executed[1].append(position.symbol)
            self.blacklist.add(position.symbol)
      else:
        #Position in long list
        if(position.side == "short"):
          #Position changed from short to long. Clear short position to prepare for long position.
          respSO = []
          tSO = threading.Thread(target = self.submitOrder, args = [abs(int(float(position.qty))), position.symbol, "buy", respSO])
          tSO.start()
          tSO.join()
        else:
          if(int(float(position.qty)) == self.qLong):
            #Position is where we want it. Pass for now.
            pass
          else:
            #Need to adjust position amount
            diff = abs(int(float(position.qty))) - self.qLong
            if(diff > 0):
              #Too many long positions. Sell some to rebalance.
              side = "sell"
            else:
              #Too little long positions. Buy some more.
              side = "buy"
            respSO = []
            tSO = threading.Thread(target = self.submitOrder, args = [abs(diff), position.symbol, side, respSO])
            tSO.start()
            tSO.join()
          executed[0].append(position.symbol)
          self.blacklist.add(position.symbol)

    #Send orders to all remaining stocks in the long and short list
    respSendBOLong = []
    tSendBOLong = threading.Thread(target=self.sendBatchOrder, args=[self.qLong, self.long, "buy", respSendBOLong])
    tSendBOLong.start()
    tSendBOLong.join()
    respSendBOLong[0][0] += executed[0]
    if(len(respSendBOLong[0][1]) > 0):
      #Handle rejected/incomplete orders and determine new quantities to purchase
      respGetTPLong = []
      tGetTPLong = threading.Thread(target = self.getTotalPrice, args = [respSendBOLong[0][0], respGetTPLong])
      tGetTPLong.start()
      tGetTPLong.join()
      if (respGetTPLong[0] > 0):
        self.adjustedQLong = self.longAmount // respGetTPLong[0]
      else:
        self.adjustedQLong = -1
    else:
      self.adjustedQLong = -1

    respSendBOShort = []
    tSendBOShort = threading.Thread(target = self.sendBatchOrder, args = [self.qShort, self.short, "sell", respSendBOShort])
    tSendBOShort.start()
    tSendBOShort.join()
    respSendBOShort[0][0] += executed[1]
    if(len(respSendBOShort[0][1]) > 0):
      #Handle rejected/incomplete orders and determine new quantities to purchase
      respGetTPShort = []
      tGetTPShort = threading.Thread(target = self.getTotalPrice, args = [respSendBOShort[0][0], respGetTPShort])
      tGetTPShort.start()
      tGetTPShort.join()
      if(respGetTPShort[0] > 0):
        self.adjustedQShort = self.shortAmount // respGetTPShort[0]
      else:
        self.adjustedQShort = -1
    else:
      self.adjustedQShort = -1

    #Reorder stocks that didn't throw an error so that the equity quota is reached
    if(self.adjustedQLong > -1):
      self.qLong = int(self.adjustedQLong - self.qLong)
      for stock in respSendBOLong[0][0]:
        respResendBOLong = []
        tResendBOLong = threading.Thread(target = self.submitOrder, args = [self.qLong, stock, "buy", respResendBOLong])
        tResendBOLong.start()
        tResendBOLong.join()

    if(self.adjustedQShort > -1):
      self.qShort = int(self.adjustedQShort - self.qShort)
      for stock in respSendBOShort[0][0]:
        respResendBOShort = []
        tResendBOShort = threading.Thread(target = self.submitOrder, args = [self.qShort, stock, "sell", respResendBOShort])
        tResendBOShort.start()
        tResendBOShort.join()

  #Re-rank all stocks to adjust longs and shorts
  def rerank(self):
    tRank = threading.Thread(target=self.rank)
    tRank.start()
    tRank.join()

    #Grabs the top and bottom quarter of the sorted stock list to get the long and short lists
    longShortAmount = len(self.allStocks) // 4
    self.long = []
    self.short = []
    for i, stockField in enumerate(self.allStocks):
      if(i < longShortAmount):
        self.short.append(stockField[0])
      elif(i > (len(self.allStocks) - 1 - longShortAmount)):
        self.long.append(stockField[0])
      else:
        continue

    #Determine amount to long/short based on total stock price of each bucket
    equity = int(float(self.alpaca.get_account().equity))

    self.shortAmount = equity * 0.30
    self.longAmount = equity - self.shortAmount

    respGetTPLong = []
    tGetTPLong = threading.Thread(target = self.getTotalPrice, args = [self.long, respGetTPLong])
    tGetTPLong.start()
    tGetTPLong.join()

    respGetTPShort = []
    tGetTPShort = threading.Thread(target = self.getTotalPrice, args = [self.short, respGetTPShort])
    tGetTPShort.start()
    tGetTPShort.join()

    self.qLong = int(self.longAmount // respGetTPLong[0])
    self.qShort = int(self.shortAmount // respGetTPShort[0])

  #Get the total price of the array of input stocks
  def getTotalPrice(self, stocks, resp):
    totalPrice = 0
    for stock in stocks:
      bars = self.alpaca.get_bars(stock, TimeFrame.Minute,
                                  pd.Timestamp('now').date(),
                                  pd.Timestamp('now').date(), limit = 1,
                                  adjustment = 'raw')

      totalPrice += bars[stock][0].c
    resp.append(totalPrice)

  #Submit a batch order that returns completed and uncompleted orders
  def sendBatchOrder(self, qty, stocks, side, resp):
    executed = []
    incomplete = []
    for stock in stocks:
      if(self.blacklist.isdisjoint({stock})):
        respSO = []
        tSubmitOrder = threading.Thread(target = self.submitOrder, args = [qty, stock, side, respSO])
        tSubmitOrder.start()
        tSubmitOrder.join()
        if(not respSO[0]):
          #Stock order did not go through, add it to incomplete
          incomplete.append(stock)
        else:
          executed.append(stock)
        respSO.clear()
    resp.append([executed, incomplete])

  #Submit an order if quantity is above 0
  def submitOrder(self, qty, stock, side, resp):
    if(qty > 0):
      try:
        self.alpaca.submit_order(stock, qty, side, "market", "day")
        print("Market order of | " + str(qty) + " " + stock + " " + side + " | completed.")
        resp.append(True)
      except:
        print("Order of | " + str(qty) + " " + stock + " " + side + " | did not go through.")
        resp.append(False)
    else:
      print("Quantity is 0, order of | " + str(qty) + " " + stock + " " + side + " | not completed.")
      resp.append(True)

  #Get percent changes of the stock prices over the past 10 minutes
  def getPercentChanges(self):
    length = 10
    for i, stock in enumerate(self.allStocks):
      bars = self.alpaca.get_bars(stock[0], TimeFrame.Minute,
                                  pd.Timestamp('now').date(),
                                  pd.Timestamp('now').date(), limit = length,
                                  adjustment = 'raw')
      self.allStocks[i][1] = (bars[stock[0]][len(bars[stock[0]]) - 1].c - bars[stock[0]][0].o) / bars[stock[0]][0].o

  #Mechanism used to rank the stocks, the basis of the Long-Short Equity Strategy
  def rank(self):
    #Ranks all stocks by percent change over the past 10 minutes (higher is better).
    tGetPC = threading.Thread(target = self.getPercentChanges)
    tGetPC.start()
    tGetPC.join()

    #Sort the stocks in place by the percent change field (marked by pc).
    self.allStocks.sort(key=lambda x: x[1])

#Run the LongShort class
ls = LongShort()
ls.run()

### **Short Selling (Short Sell)**

In [ ]:
#Import libraries
import alpaca_trade_api as tradeapi
import time

if __name__ == '__main__':
  """
  With the Alpaca API, you can open a short position by submitting a sell
  order for a security that you have no open long position with.
  """

  #First, open the API connection.
  api = tradeapi.REST('PKWNGPNK1B3QSSSWQYNZ', 'Ny8OOlvzdF57HC3sV6VOd3EM09XuuhT7owIy67W9', 'https://paper-api.alpaca.markets', api_version = 'v2')

  #The security we'll be shorting.
  symbol = 'AAPL'

  #Submit a market order to open a short position of one share
  order = api.submit_order(symbol, 1, 'sell', 'market', 'day')
  print("Market order submitted.")

  #Submit a limit order to attempt to grow our short position
  #First, get an up-to-date price for our symbol.
  symbol_bars = api.get_barset(symbol, 'minute', 1).df.iloc[0]
  symbol_price = symbol_bars[symbol]['close']
  #Submit an order for one share at that price
  order = api.submit_order(symbol, 1, 'sell', 'limit', 'day', symbol_price)
  print("Limit order submitted.")

  #Wait a second for our orders to fill...
  print('Waiting...')
  time.sleep(1)

  #Check on our position
  position = api.get_position(symbol)
  if int(position.qty) < 0:
      print(f'Short position open for {symbol}')

### **Sentiment Analysis**

In [ ]:
#Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.downloader.download('vader_lexicon')

#Parameters
n = 5 #The # of article headlines displayed per ticker
tickers = ['TSLA', 'FB', 'AAPL', 'LYFT', 'COIN', 'RIOT']

#Get data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url = url, headers = {'user-agent': 'my-app/0.0.1'})
    resp = urlopen(req)
    html = BeautifulSoup(resp, features = "lxml")
    news_table = html.find(id = 'news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
        print('\nRecent News Headlines for {}: '.format(ticker))
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text, '(',td_text,')') #Headlines news
            if i == n-1:
                break
except KeyError:
    pass

#Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text()
        date_scrape = x.td.text.split()
        if len(date_scrape) == 1:
            time = date_scrape[0]
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        ticker = file_name.split('_')[0]
        parsed_news.append([ticker, date, time, text])

#Sentiment analysis
analyzer = SentimentIntensityAnalyzer()
columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns = columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()
df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix = '_right')

#View Data
news['Date'] = pd.to_datetime(news.Date).dt.date
unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}
values = []
for ticker in tickers:
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print('\n')
    print(dataframe.head()) #Tickers sentiment

    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)

df = pd.DataFrame(list(zip(tickers, values)), columns = ['Ticker', 'Mean Sentiment'])
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending = False)
print('\n')
print(df) #Sentiment stocks

## **ETF Stocks**

### **Sentiment & EMA Trading Strategy**

In [ ]:
#Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.downloader.download('vader_lexicon')

#Parameters
n = 3 #The # of article headlines displayed per ticker
tickers = ['QQQ', 'SPY', 'VOO', 'VTI', 'BLOK', #Popular
           'IYZ', 'IGN', 'XTL', 'EWCO', #Communications Equities
           'XLE', 'VDE', 'XOP', 'IYE', 'OIH', 'IXC', 'FENY', 'FTXN', #Energy Equities
           'XLF', 'VFH', 'KRE', 'KBE', 'KBWB', 'IYG', 'IYF', 'IXG', 'EUFN', 'FNCL', 'FXO', 'IAT', #Financials Equities
           'XLV', 'VHT', 'IBB', 'IHI', 'XBI', 'ARKG', 'IXJ', 'IYH', 'FHLC', 'FBT', 'FXH', 'IHF', #Health & Biotech Equities
           'XLI', 'VIS', 'JETS', 'ITA', 'FXR', 'IYJ', 'FTXR', 'XAR', #Industrials Equities
           'VGT', 'XLK', 'SOXX', 'SMH', 'IYW', 'FTEC', 'SKYY', 'IGV', 'CIBR', 'IXN', #Technology Equities
           'IYT', 'XTN', 'BOAT', 'MOTO', #Transportation Equities
           'XLU', 'VPU', 'PAVE', 'IGF', 'NFRA', 'FUTY' #Utilities Equities
          ]

#Get data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}
for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url = url, headers = {'user-agent': 'my-app/0.0.1'})
    resp = urlopen(req)
    html = BeautifulSoup(resp, features = "lxml")
    news_table = html.find(id = 'news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
        #print('\nRecent News Headlines for {}: '.format(ticker))
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            #print(a_text,'(',td_text,')') #Headlines news
            if i == n-1:
                break
except KeyError:
    pass

#Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text()
        date_scrape = x.td.text.split()
        if len(date_scrape) == 1:
            time = date_scrape[0]
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        ticker = file_name.split('_')[0]
        parsed_news.append([ticker, date, time, text])

#Sentiment analysis
analyzer = SentimentIntensityAnalyzer()
columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns = columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()
df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix = '_right')

#View data
news['Date'] = pd.to_datetime(news.Date).dt.date
unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}
values = []
for ticker in tickers:
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    #print(dataframe.head(3)) #Tickers sentiment
    #print('\n')

    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)

df = pd.DataFrame(list(zip(tickers, values)), columns = ['Ticker', 'Mean Sentiment'])
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending = False)
print(df) #Sentiment stocks

# print("\nTop 3 Stocks to Invest")
# top3 = df.nlargest(n = 3, columns = ['Mean Sentiment'])
# print(top3)
# index_toplist = top3.index.tolist()
# print(index_toplist)

# print("\nBottom 3 Stocks to Invest")
# bottom3 = df.nsmallest(n = 3, columns = ['Mean Sentiment'])
# print(bottom3)
# index_bottomlist = bottom3.index.tolist()
# print(index_bottomlist)

print("\nTop Stock to Invest")
top_stock = df.nlargest(n = 1, columns = ['Mean Sentiment'])
print(top_stock)
symb_list = top_stock.index.tolist()
symbs = symb_list[0]

In [ ]:
import alpaca_trade_api as tradeapi
import numpy as np
import pandas as pd
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#Authentication and connection details
api_key = 'PKSK1QYR2O532NSHT0KU'
api_secret = '7BNhzgktxgEwG8ULs1e8ZOCtDDiWQCgW11RzWwkk'
base_url = 'https://paper-api.alpaca.markets'

#Instantiate REST API
api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')

#Get our account information
account = api.get_account()

#The mail addresses and password
sender_address = 'fintechtraininggroup1@gmail.com'
sender_pass = 'FintechGroup1'
receiver_address = 'fintechtraininggroup1@gmail.com'

#Setup the MIME
message = MIMEMultipart()
message['From'] = 'Trading Bot'
message['To'] = receiver_address
message['Subject'] = 'Fintech Python Trading Bot'

#Selection of stock
symb = symbs
stock_price = []
pos_held = False
mail_content= ""
mail_list = []

#Get market closing price within 10 minutes
def get_market_data():
  minute = 10
  symb = symbs
  market_data = api.get_barset(symb, 'minute', limit = minute)
  bars = market_data[symb]

  close_list = []
  for bar in market_data[symb]:
      close_list.append(bar.c)

  close_list = np.array(close_list, dtype=np.float64)

  return close_list

#Calculate EMA
def calculate_ema(prices, days, smoothing = 2):
  ema = [sum(prices[:days]) / days]
  for price in prices[days:]:
      ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
  return ema

#Email notification
def mailSending(mail_content):
  #The body and the attachments for the mail
  message.attach(MIMEText(mail_content, 'plain'))

  #Create SMTP session for sending the mail
  session = smtplib.SMTP('smtp.gmail.com', 587) #Use gmail with port
  session.starttls() #enable security
  session.login(sender_address, sender_pass) #Login with mail_id and password
  text = message.as_string()
  session.sendmail(sender_address, receiver_address, text)
  session.quit()
  print('Mail Sent')

#Trading algorithm
clock = api.get_clock()
print('The market is {}'.format('open.' if clock.is_open else 'closed.'))

while True:
  print("")
  print("Checking Price")
  ema = calculate_ema(get_market_data(), len(get_market_data()))[-1]
  last_price = get_market_data()[-1] #Most recent closing price
  print("Equity: {0}".format(account.equity))
  print("Exponential Moving Average: " + str(ema))
  print("Last Price: " + str(last_price))

  if clock.is_open == True:
    #Buy
    if last_price > ema and not pos_held: #If price is crossing EMA, and we haven't already bought -> so we buy it
      print("Buy")
      api.submit_order(
        symbol = symb,
        qty = (0.2 * float(account.equity)) // last_price,
        side = 'buy',
        type = 'market',
        time_in_force = 'gtc'
      )
      mail_content = "Trades have been made. You have buy the stocks. \n"
      pos_held = True
      stock_price.append(last_price)
    print(stock_price)

    #Sell
    if len(stock_price) != 0:
      if last_price >= (stock_price[-1] + stock_price[-1] * 0.002) and pos_held:
        print("Sell")
        api.submit_order(
          symbol = symb,
          qty = (0.2 * float(account.equity)) // last_price,
          side = 'sell',
          type = 'market',
          time_in_force = 'gtc'
        )
        mail_content = "Trades have been made. You have sell the stocks and take profit. \n"
        pos_held = False
        stock_price.clear()
      elif last_price <= (stock_price[-1] - stock_price[-1] * 0.005) and pos_held:
        print("Sell")
        api.submit_order(
          symbol = symb,
          qty = (0.2 * float(account.equity)) // last_price,
          side = 'sell',
          type = 'market',
          time_in_force = 'day'
        )
        mail_content = "Trades have been made. You have sell the stocks and stop loss. \n"
        pos_held = False
        stock_price.clear()
  else:
    mail_content = "The market is closed. \n"

  #Condition for sending email that should be running only once for each
  if (mail_list.count(mail_content) < 1):
    mailSending(mail_content)
    if (mail_content not in mail_list):
      mail_list.clear() #Clear after send
    mail_list.append(mail_content)  #Append back what we go through

  time.sleep(60) #Refresh every minute

### **Sentiment & EMA Trading Strategy (Blockchain ETFs)**

In [ ]:
#Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.downloader.download('vader_lexicon')

#Parameters
n = 3 #The # of article headlines displayed per ticker
# tickers = ['QQQ', 'SPY', 'VOO', 'VTI', #Popular
#            'BITW', 'BITS', 'BITO', 'GBTC', 'ETHE', 'BTF', 'XBTF', 'VBB',  #Cryptocurrency ETFs
#            'BITQ', 'BKCH', 'BLCN', 'BLOK', 'CRPT', 'DAPP', 'KOIN', 'LEGR', #Blockchain ETFs
#            'IYZ', 'IGN', 'XTL', 'EWCO', #Communications Equities
#            'XLE', 'VDE', 'XOP', 'IYE', 'OIH', 'IXC', 'FENY', 'FTXN', #Energy Equities
#            'XLF', 'VFH', 'KRE', 'KBE', 'KBWB', 'IYG', 'IYF', 'IXG', 'EUFN', 'FNCL', 'FXO', 'IAT', #Financials Equities
#            'XLV', 'VHT', 'IBB', 'IHI', 'XBI', 'ARKG', 'IXJ', 'IYH', 'FHLC', 'FBT', 'FXH', 'IHF', #Health & Biotech Equities
#            'XLI', 'VIS', 'JETS', 'ITA', 'FXR', 'IYJ', 'FTXR', 'XAR', #Industrials Equities
#            'VGT', 'XLK', 'SOXX', 'SMH', 'IYW', 'FTEC', 'SKYY', 'IGV', 'CIBR', 'IXN', #Technology Equities
#            'IYT', 'XTN', 'BOAT', 'MOTO', #Transportation Equities
#            'XLU', 'VPU', 'PAVE', 'IGF', 'NFRA', 'FUTY' #Utilities Equities
#           ]
tickers = ['BITO', 'BKCH', 'BLCN', 'BLOK', 'DAPP', 'KOIN', 'LEGR']

#Get data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}
for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url = url, headers = {'user-agent': 'my-app/0.0.1'})
    resp = urlopen(req)
    html = BeautifulSoup(resp, features = "lxml")
    news_table = html.find(id = 'news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
        #print('\nRecent News Headlines for {}: '.format(ticker))
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            #print(a_text,'(',td_text,')') #Headlines news
            if i == n-1:
                break
except KeyError:
    pass

#Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text()
        date_scrape = x.td.text.split()
        if len(date_scrape) == 1:
            time = date_scrape[0]
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        ticker = file_name.split('_')[0]
        parsed_news.append([ticker, date, time, text])

#Sentiment analysis
analyzer = SentimentIntensityAnalyzer()
columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns = columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()
df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix = '_right')

#View data
news['Date'] = pd.to_datetime(news.Date).dt.date
unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}
values = []
for ticker in tickers:
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    #print(dataframe.head(3)) #Tickers sentiment
    #print('\n')

    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)

df = pd.DataFrame(list(zip(tickers, values)), columns = ['Ticker', 'Mean Sentiment'])
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending = False)
print(df) #Sentiment stocks

# print("\nTop 3 Stocks to Invest")
# top3 = df.nlargest(n = 3, columns = ['Mean Sentiment'])
# print(top3)
# index_toplist = top3.index.tolist()
# print(index_toplist)

# print("\nBottom 3 Stocks to Invest")
# bottom3 = df.nsmallest(n = 3, columns = ['Mean Sentiment'])
# print(bottom3)
# index_bottomlist = bottom3.index.tolist()
# print(index_bottomlist)

print("\nTop Stock to Invest")
top_stock = df.nlargest(n = 1, columns = ['Mean Sentiment'])
print(top_stock)
symb_list = top_stock.index.tolist()
symbs = symb_list[0]

In [ ]:
import alpaca_trade_api as tradeapi
import numpy as np
import pandas as pd
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#Authentication and connection details
api_key = 'PKSK1QYR2O532NSHT0KU'
api_secret = '7BNhzgktxgEwG8ULs1e8ZOCtDDiWQCgW11RzWwkk'
base_url = 'https://paper-api.alpaca.markets'

#Instantiate REST API
api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')

#Get our account information
account = api.get_account()

#The mail addresses and password
sender_address = 'fintechtraininggroup1@gmail.com'
sender_pass = 'FintechGroup1'
receiver_address = 'fintechtraininggroup1@gmail.com'

#Setup the MIME
message = MIMEMultipart()
message['From'] = 'Trading Bot'
message['To'] = receiver_address
message['Subject'] = 'Fintech Python Trading Bot'

#Selection of stock
symb = symbs
stock_price = []
pos_held = False
mail_content= ""
mail_list = []

#Get market closing price within 10 minutes
def get_market_data():
  minute = 10
  symb = symbs
  market_data = api.get_barset(symb, 'minute', limit = minute)
  bars = market_data[symb]

  close_list = []
  for bar in market_data[symb]:
      close_list.append(bar.c)

  close_list = np.array(close_list, dtype=np.float64)

  return close_list

#Calculate EMA
def calculate_ema(prices, days, smoothing = 2):
  ema = [sum(prices[:days]) / days]
  for price in prices[days:]:
      ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
  return ema

#Email notification
def mailSending(mail_content):
  #The body and the attachments for the mail
  message.attach(MIMEText(mail_content, 'plain'))

  #Create SMTP session for sending the mail
  session = smtplib.SMTP('smtp.gmail.com', 587) #Use gmail with port
  session.starttls() #enable security
  session.login(sender_address, sender_pass) #Login with mail_id and password
  text = message.as_string()
  session.sendmail(sender_address, receiver_address, text)
  session.quit()
  print('Mail Sent')

#Trading algorithm
clock = api.get_clock()
print('The market is {}'.format('open.' if clock.is_open else 'closed.'))

while True:
  print("")
  print("Checking Price")
  ema = calculate_ema(get_market_data(), len(get_market_data()))[-1]
  last_price = get_market_data()[-1] #Most recent closing price
  print("Equity: {0}".format(account.equity))
  print("Exponential Moving Average: " + str(ema))
  print("Last Price: " + str(last_price))

  if clock.is_open == True:
    #Buy
    if last_price > ema and not pos_held: #If price is crossing EMA, and we haven't already bought -> so we buy it
      print("Buy")
      api.submit_order(
        symbol = symb,
        qty = (0.2 * float(account.equity)) // last_price,
        side = 'buy',
        type = 'market',
        time_in_force = 'gtc'
      )
      mail_content = "Trades have been made. You have buy the stocks. \n"
      pos_held = True
      stock_price.append(last_price)
    print(stock_price)

    #Sell
    if len(stock_price) != 0:
      if last_price >= (stock_price[-1] + stock_price[-1] * 0.002) and pos_held:
        print("Sell")
        api.submit_order(
          symbol = symb,
          qty = (0.2 * float(account.equity)) // last_price,
          side = 'sell',
          type = 'market',
          time_in_force = 'gtc'
        )
        mail_content = "Trades have been made. You have sell the stocks and take profit. \n"
        pos_held = False
        stock_price.clear()
      elif last_price <= (stock_price[-1] - stock_price[-1] * 0.005) and pos_held:
        print("Sell")
        api.submit_order(
          symbol = symb,
          qty = (0.2 * float(account.equity)) // last_price,
          side = 'sell',
          type = 'market',
          time_in_force = 'day'
        )
        mail_content = "Trades have been made. You have sell the stocks and stop loss. \n"
        pos_held = False
        stock_price.clear()
  else:
    mail_content = "The market is closed. \n"

  #Condition for sending email that should be running only once for each
  if (mail_list.count(mail_content) < 1):
    mailSending(mail_content)
    if (mail_content not in mail_list):
      mail_list.clear() #Clear after send
    mail_list.append(mail_content)  #Append back what we go through

  time.sleep(60) #Refresh every minute

# **Binance**

In [ ]:
!pip install python-binance
!pip install binance-futures-connector
!pip install requests
!pip install talib-binary

     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 103 kB 33.9 MB/s 
     |████████████████████████████████| 288 kB 63.3 MB/s 
     |████████████████████████████████| 1.1 MB 55.2 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 144 kB 26.6 MB/s 
     |████████████████████████████████| 271 kB 61.4 MB/s 
     |████████████████████████████████| 486 kB 8.2 MB/s 
     |████████████████████████████████| 3.1 MB 45.2 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 61 kB 7.6 MB/s 
     |████████████████████████████████| 74 kB 3.1 MB/s 
     |████████████████████████████████| 3.6 MB 33.1 MB/s 
     |████████████████████████████████| 251 kB 47.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstal

## **EMA Trading Strategy**

### **Binance Future (Connector)**

In [ ]:
#Import libraries
from binance.futures import Futures

#Binance Futures API
client = Futures(key = '0727db89ccf6d53b11ac071fdd346af9593c125c71df40cca50726514f2cf39c', secret = '54db514780988524fe5832c8b469ef40a46627e6c3900aa8d3ebbcdf306ce788')

#Get our account information
print(client.account())

client = Futures()
print(client.time())

#Post a new order
params = {
    'symbol': 'BTCUSDT',
    'side': 'SELL',
    'type': 'LIMIT',
    'timeInForce': 'GTC',
    'quantity': 0.002,
    'price': 59808
}

# response = client.new_order(**params)
# print(response)

ClientError: ignored

### **Binance Testnet (Futures Testnet)**

https://algotrading101.com/learn/binance-python-api-guide/

In [ ]:
#Import libraries
import requests
from binance.client import Client
import pandas as pd
import numpy as np
import talib #talib library for various calculation related to financial and technical indicators
import time
#Binance Testnet API
API_Key = "0727db89ccf6d53b11ac071fdd346af9593c125c71df40cca50726514f2cf39c"
API_Secret = "54db514780988524fe5832c8b469ef40a46627e6c3900aa8d3ebbcdf306ce788"

client = Client(API_Key, API_Secret, testnet = True)

#Constants
SYMBOL = "BTCUSDT"
TIME_PERIOD= "15m" #Taking 15 minutes time period
LIMIT = "200" #Taking 200 candles as limit
QNTY = 1
stock_price = []
pos_held = False

#Get our information
account = client.futures_account_balance()
print(account)
asset = pd.DataFrame(account)
print(asset)
last_price = client.get_symbol_ticker(symbol = "BTCUSDT")
print(last_price)

#Get data from Binance
def get_data():
    url = "https://api.binance.com/api/v3/klines?symbol={}&interval={}&limit={}".format(SYMBOL, TIME_PERIOD, LIMIT)
    res = requests.get(url)
    return_data = []
    for each in res.json():
        return_data.append(float(each[4]))
    return np.array(return_data)

#Calculate EMA
def calculate_ema(prices, days, smoothing = 2):
  ema = [sum(prices[:days]) / days]
  for price in prices[days:]:
      ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
  return ema

#EMA Algorithm
while True:
  print("")
  print("Checking Price")
  ema = calculate_ema(get_data(), len(get_data()))[-1]
  last_price = get_data()[-1] #Most recent closing price
  print("Exponential Moving Average: " + str(ema))
  print("Last Price: " + str(last_price))

  #Buy
  if last_price > ema and not pos_held: #If price is crossing EMA, and we haven't already bought -> so we buy it
    print("Buy")
    client.futures_create_order(symbol = SYMBOL, side = "BUY", type = "MARKET", quantity = QNTY)
    pos_held = True
    stock_price.append(last_price)
  print(stock_price)

  #Sell
  if len(stock_price) != 0:
    if last_price >= (stock_price[-1] + stock_price[-1] * 0.2) and pos_held:
      print("Sell take profit")
      client.futures_create_order(symbol = SYMBOL, side = "SELL", type = "MARKET", quantity = QNTY)
      pos_held = False
      stock_price.clear()
    elif last_price <= (stock_price[-1] - stock_price[-1] * 0.1) and pos_held:
      print("Sell stop loss")
      client.futures_create_order(symbol = SYMBOL, side = "SELL", type = "MARKET", quantity = QNTY)
      pos_held = False
      stock_price.clear()
    time.sleep(10) #Refresh every minute

[{'accountAlias': 'oCSgSgTisRAu', 'asset': 'BNB', 'balance': '2.54950707', 'withdrawAvailable': '2.54950707', 'updateTime': 1648030225626}, {'accountAlias': 'oCSgSgTisRAu', 'asset': 'USDT', 'balance': '99992.91474800', 'withdrawAvailable': '97882.85272209', 'updateTime': 1647663701633}, {'accountAlias': 'oCSgSgTisRAu', 'asset': 'BUSD', 'balance': '1000.40016006', 'withdrawAvailable': '1000.40016006', 'updateTime': 1647597839601}]
   accountAlias asset         balance withdrawAvailable     updateTime
0  oCSgSgTisRAu   BNB      2.54950707        2.54950707  1648030225626
1  oCSgSgTisRAu  USDT  99992.91474800    97882.85272209  1647663701633
2  oCSgSgTisRAu  BUSD   1000.40016006     1000.40016006  1647597839601
{'symbol': 'BTCUSDT', 'price': '42145.04000000'}

Checking Price
Exponential Moving Average: 41978.14720000001
Last Price: 42148.94
Buy
[42148.94]

Checking Price
Exponential Moving Average: 41978.16335000001
Last Price: 42152.17
[42148.94]

Checking Price
Exponential Moving Averag

KeyboardInterrupt: ignored

### **Binance Vision (Spot Testnet)**

In [ ]:
#Import libraries
from binance.client import Client
import pandas as pd

#Binance Vision API
API_Key = "Input your API Key"
API_Secret = "Input your API Secret"
API_URL = 'https://testnet.binancefuture.com'

client = Client(API_Key, API_Secret, testnet = True)
client.API_URL = 'https://testnet.binance.vision/api'

#Get our account information
print(client.get_server_time())
print(client.get_exchange_info())
print(client.get_symbol_info('BTCUSDT'))
print(client.get_all_tickers())

info = client.get_account()
print(info)

df = pd.DataFrame(info["balances"])
df["free"] = df["free"].astype(float)
df = df[df["free"] > 0]
print(df)

#Buy or Sell
client.create_order(symbol = "BTCUSDT", quantity = 0.01, type = "MARKET", side = "SELL")

open_orders = client.get_open_orders(symbol = "BTCUSDT")
print("open orders BTCUSDT  :",open_orders)

open_order = client.get_open_orders()
print("open orders :",open_order)

{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'updateTime': 1647230768933, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '0.93000000', 'locked': '0.00000000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '12670.26763074', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}
{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'updateTime': 1647230768933, 'accountType': 'SPOT', 'balances': [{'as